In [34]:
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import base64
import time
from bs4 import BeautifulSoup
from urllib.parse import quote

In [79]:
ano = 2014

df_edicao = pd.read_csv(f'conceito_enade_{ano}_Teste.csv')
codigo_cursos = df_edicao["Codigo_do_Curso"].unique().tolist()

dict_codigo_ce = {
    codigo: 0
    for codigo
    in codigo_cursos
}

In [80]:
async def fetch(session, codigo, timeout):
    utf8_codigo_curso = str(codigo).encode("utf-8")
    base64_codigo_curso = base64.b64encode(utf8_codigo_curso).decode("utf-8")
    base64_utf8_codigo_curso = quote(base64_codigo_curso, encoding="utf-8")
    #print(base64_utf8_codigo_curso)
        
    async with session.get(f'https://emec.mec.gov.br/emec/consulta-curso/listar-historico-indicadores-curso/c1999930082674af6577f0c513f05a96/{base64_utf8_codigo_curso}/list/10', 
                           timeout=timeout) as response:
        html_table = await response.text()
        soup = BeautifulSoup(html_table, 'html.parser')

        rows = soup.find('tbody').find_all('tr')

        row_2014 = None
        for row in rows:
            if row.find('td', text='2014'):
                row_2014 = row
                break

        if row_2014:
            enade_2014 = row_2014.find_all('td')[1].text
            dict_codigo_ce[codigo] = int(enade_2014)

async def fetch_all():
    result = []
    timeout = aiohttp.ClientTimeout(total=120)
    conn = aiohttp.TCPConnector(limit=10, limit_per_host=10)

    async with aiohttp.ClientSession(timeout=timeout, connector=conn) as session:
        await asyncio.gather(*[fetch(session, codigo, timeout) for codigo in codigo_cursos], return_exceptions=True)
        #return result

In [81]:
await fetch_all()

In [84]:
len(dict_codigo_ce)

1505

In [83]:
print(dict_codigo_ce)

{36: 4, 127: 4, 317: 4, 513: 4, 594: 5, 614: 5, 631: 3, 703: 5, 778: 3, 865: 3, 872: 3, 968: 2, 1051: 4, 1110: 4, 1261: 4, 1452: 4, 1529: 2, 1532: 1, 1660: 3, 1721: 2, 1904: 4, 2090: 3, 2209: 4, 2310: 3, 2712: 4, 2717: 4, 3190: 3, 3193: 5, 3197: 3, 3211: 3, 3419: 3, 3531: 2, 3676: 3, 3858: 2, 3859: 2, 3935: 3, 3952: 3, 4547: 3, 4675: 2, 4963: 3, 5079: 2, 5238: 3, 5485: 3, 5500: 4, 5841: 3, 6491: 5, 6582: 2, 6668: 2, 6756: 3, 6847: 2, 6852: 3, 7142: 2, 7292: 3, 7301: 3, 7511: 4, 7523: 4, 7839: 4, 8170: 3, 8340: 2, 8913: 4, 9302: 2, 9311: 2, 9414: 3, 9425: 2, 9440: 4, 9470: 1, 9586: 2, 10059: 3, 10077: 2, 10159: 2, 10348: 2, 10515: 2, 10748: 2, 10814: 3, 10978: 1, 10999: 2, 11238: 3, 11452: 3, 11554: 4, 11730: 3, 12025: 3, 12313: 4, 12581: 4, 12710: 4, 12836: 4, 12837: 3, 12946: 5, 13216: 4, 13277: 5, 13401: 5, 13446: 4, 13595: 5, 13717: 5, 13881: 4, 13980: 5, 14098: 5, 14217: 5, 14647: 5, 14707: 4, 14722: 4, 15002: 4, 15103: 3, 15127: 3, 15262: 3, 15264: 3, 15273: 3, 15490: 2, 15543: 2,